In [15]:
from pyspark.sql import SparkSession   
from pyspark.sql.functions import col, when
from pyspark.sql.types import StructType, StructField, \
  StringType, FloatType, DateType
from pyspark.sql.functions import max, avg, month, year, round

spark = SparkSession.builder \
  .appName('Weather') \
  .getOrCreate()

schema = StructType([
  StructField('station_id', StringType(), False),
  StructField('date', DateType(), False),
  StructField('temperature', FloatType(), True),
  StructField('precipitation', FloatType(), True),
  StructField('wind_speed', FloatType(), True)
])

df = spark.read.csv('weather/weather_data.csv', 
                    header=True, schema=schema)

# Общая информация о наборе данных
"""
# print('Схема данных:')
df.printSchema()

# print('Первые пару записей:')
df.show(5)

# Вывод невалидных записей
df_empty_values = (
  df.filter(df['temperature'].isNull() | df['precipitation'].isNull() | df['wind_speed'].isNull())
)
print(f'Записи с пропусками и их количество: {df_empty_values.count()}')
df_empty_values.show()
"""
# Избавление от пустых записей
for column in ['temperature', 'precipitation', 'wind_speed']:
  df = (
    df.withColumn(column,
      when(col(column).isNull(), df.select(avg(col(column))).collect()[0][0]) 
        .otherwise(col(column))) 
      .withColumn(column, round(column, 3))
  )
  
# 1
print('Топ-5 самых жарких дней за все время наблюдений')
(
  df.orderBy(col('temperature').desc())
)[['date', 'temperature']].show(5)

# 2
last_year = df.select(max(col('date'))).collect()[0][0].year
print(f'Метеостанция с наибольшим количеством осадков за последний год ({last_year})')
(
  df.filter(year('date') == last_year) 
    .groupBy('station_id').agg({'precipitation': 'sum'}) 
    .withColumnRenamed('sum(precipitation)', 'total precipitation') 
    .withColumn('total precipitation', round(col('total precipitation'), 5))
    .orderBy(col('total precipitation').desc()) 
).show(1)

# 3
print('Средняя температура по месяцам за все время наблюдений')
(
  df.groupBy(month('date').alias('month')) 
    .avg('temperature').orderBy('month') 
    .withColumnRenamed('avg(temperature)', 'avg temp') 
    .withColumn('avg temp', round(col('avg temp'), 5))
).show()

spark.stop()

Топ-5 самых жарких дней за все время наблюдений
+----------+-----------+
|      date|temperature|
+----------+-----------+
|2021-08-20|     39.983|
|2023-12-02|     39.968|
|2022-03-28|     39.825|
|2019-02-11|     39.767|
|2020-06-10|     39.691|
+----------+-----------+
only showing top 5 rows

Метеостанция с наибольшим количеством осадков за последний год (2023)
+----------+-------------------+
|station_id|total precipitation|
+----------+-------------------+
| station_5|            642.928|
+----------+-------------------+
only showing top 1 row

Средняя температура по месяцам за все время наблюдений
+-----+--------+
|month|avg temp|
+-----+--------+
|    1|11.35655|
|    2| 9.06724|
|    3| 7.24406|
|    4|12.02455|
|    5| 9.90289|
|    6|13.42109|
|    7|  6.1857|
|    8|10.96781|
|    9| 9.59674|
|   10| 9.09881|
|   11| 7.26582|
|   12|11.21853|
+-----+--------+

